In [24]:
# load the archive data file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

img= Image.open("archive/s1/1.pgm")
np_img = np.array(img)
 
Data = np.zeros((400,10304))
cnt=0
#  open archive directory and read file s1
import os
path = "archive/s1"
files = os.listdir(path)
for file in files:
    img = Image.open(path + "/" + file)
    np_img = np.array(img)
    np_img = np_img.flatten()
    print(np_img)
    Data[cnt]=np_img
    cnt+=1
    print(np_img.shape)

print(Data)    

[48 49 45 ... 47 46 46]
(10304,)
[34 34 33 ... 37 40 33]
(10304,)
[60 60 62 ... 32 34 34]
(10304,)
[39 44 53 ... 29 26 29]
(10304,)
[63 53 35 ... 41 10 24]
(10304,)
[64 76 80 ... 35 37 39]
(10304,)
[ 43  50  41 ... 158 153 169]
(10304,)
[41 47 47 ... 35 37 38]
(10304,)
[44 43 32 ... 43 43 37]
(10304,)
[42 41 44 ... 42 43 41]
(10304,)
[[48. 49. 45. ... 47. 46. 46.]
 [34. 34. 33. ... 37. 40. 33.]
 [60. 60. 62. ... 32. 34. 34.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
